In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import urllib3.request
import requests; print("Requests", requests.__version__)
import pandas as pd; print("Pandas", pd.__version__)
import numpy as np; print("Numpy", np.__version__)
import nltk; print("NLTK", nltk.__version__)
import re; print("Re", re.__version__)
import json; print("Pandas", pd.__version__)
import pprint
from collections import defaultdict
import os
import nltk; print("nltk", nltk.__version__)
import spacy; print("spacy", spacy.__version__)
import unidecode
import unicodedata
import pattern; print ("pattern", pattern.__version__)
import string
import time

print (os.environ['CONDA_DEFAULT_ENV'])

/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


Darwin-18.7.0-x86_64-i386-64bit
Python 3.7.2 (default, Dec 29 2018, 00:00:04) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Requests 2.19.1
Pandas 0.24.2
Numpy 1.16.4
NLTK 3.3
Re 2.2.1
Pandas 0.24.2
nltk 3.3
spacy 2.1.4
pattern 3.6
base


## Import Job Posting Files

In [2]:
data = pd.read_csv('datascientist.csv')
print(data.head())
print(len(data))

   Unnamed: 0                  Title            Location           City State  \
0           0         Data Scientist       Cambridge, MA      Cambridge    MA   
1           2         Data Scientist        Berkeley, CA       Berkeley    CA   
2           3  Senior Data Scientist  Berkeley, CA 94710       Berkeley    CA   
3           4  Senior Data Scientist        Berkeley, CA       Berkeley    CA   
4           5         Data Scientist   Beverly Hills, CA  Beverly Hills    CA   

          Zip     Country                                     Qualifications  \
0  None Found  None Found                                         None Found   
1  None Found  None Found  \nProficiency in Python, including both data s...   
2       94710  None Found                                         None Found   
3  None Found  None Found                                         None Found   
4  None Found  None Found                                         None Found   

                                

In [3]:
df = data[:25]

### Remove line breaks and special symbols

In [4]:
# Remove line breaks and '\n'
for n in range (0, len(df)) :
    data.FullDescriptions[n] = df.FullDescriptions[n].replace('\n', ' ').replace('*', ' ').replace(':', ' ').replace('#', ' ')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
df

,Unnamed: 0,Title,Location,City,State,Zip,Country,Qualifications,Skills,Responsibilities,Education,Requirement,FullDescriptions
0,0,Data Scientist,"Cambridge, MA",Cambridge,MA,None Found,None Found,None Found,None Found,None Found,None Found,None Found,------ ------ ------------------------------...
1,2,Data Scientist,"Berkeley, CA",Berkeley,CA,None Found,None Found,"\nProficiency in Python, including both data s...",None Found,\nHelp develop KoBold’s proprietary software c...,None Found,None Found,About the company KoBold Metals is using machi...
2,3,Senior Data Scientist,"Berkeley, CA 94710",Berkeley,CA,94710,None Found,None Found,\nPh.D. in a quantitative discipline plus 2+ y...,\nOwn the development of predictive analytics ...,None Found,None Found,About Pivot Bio --------------- Fueled by an ...
3,4,Senior Data Scientist,"Berkeley, CA",Berkeley,CA,None Found,None Found,None Found,None Found,None Found,None Found,None Found,Senior Data Scientist Senior Data Scientists o...
4,5,Data Scientist,"Beverly Hills, CA",Beverly Hills,CA,None Found,None Found,None Found,None Found,None Found,None Found,None Found,"Fandango is seeking a driven, analytically cur..."
5,6,Data Scientist,"New York, NY",New York,NY,None Found,None Found,None Found,None Found,None Found,None Found,None Found,-------- About Us -------- KeyMe is a New Yor...
6,10,Senior Data Scientist,"Chicago, IL",Chicago,IL,None Found,None Found,None Found,None Found,None Found,None Found,None Found,----------------------------------------------...
7,12,Data Scientist,"Brisbane, CA 94005",Brisbane,CA,94005,None Found,"Ph.D in Physics, Computational Biology or Comp...",None Found,Help design and produce analytical data pipeli...,None Found,None Found,"Headquartered in Brisbane, Calif., CareDx, Inc..."
8,13,Artificial Intelligence/Analytics Data Scienti...,"Burbank, CA",Burbank,CA,None Found,None Found,"Qualifications: 1 – 3 years experience, Bachel...",None Found,None Found,None Found,None Found,Candidates should be flexible / willing to wor...
9,14,Data Scientist Manager - Hiring in Burbank!,"Burbank, CA",Burbank,CA,None Found,None Found,Qualifications: 7 – 10 (3 years min relevant e...,None Found,None Found,None Found,None Found,5-10 years professional work experience as a d...


### Universal Sentence Encoder - one corpus at a time - TAKES 30 SECONDS PER corpus

In [6]:
tempCorpus = df['FullDescriptions']
dummy = []

In [7]:
with tf.Graph().as_default():
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    
    for corpus in tempCorpus :
        messages = [corpus]
        output = embed(messages)
 
        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            t1 = time.time()
            message_embeddings = session.run(output)
            print(time.time() - t1)
            dummy.append(message_embeddings)

Instructions for updating:
Colocations handled automatically by placer.


W1011 10:50:15.862906 4466746816 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1011 10:50:18.517322 4466746816 saver.py:1483] Saver not created because there are no variables in the graph to restore


2.004863977432251
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

15.304362058639526
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1011 11:12:02.695706 4466746816 saver.py:1483] Saver not created because there are no variables in the graph to restore


15.818544864654541
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1011 11:13:25.844139 4466746816 saver.py:1483] Saver not created because there are no variables in the graph to restore


19.881635189056396


In [8]:
dummy[0]

array([[-6.02395125e-02,  1.60463080e-02, -2.23891102e-02,
        -4.34342166e-03,  6.44785399e-03, -9.03625861e-02,
         1.22285234e-02,  2.78480612e-02, -2.59086229e-02,
         3.79698128e-02, -1.95618309e-02,  5.08786775e-02,
        -5.15983067e-02,  5.73120825e-02,  9.76173300e-03,
         2.70774812e-02,  2.05946229e-02, -9.61376820e-03,
         9.41379368e-02, -2.22235490e-02,  6.18627928e-02,
        -3.83428335e-02, -5.98125495e-02,  9.38614011e-02,
        -4.54228278e-03,  5.61258681e-02, -3.17748524e-02,
        -2.19284613e-02,  3.10339388e-02, -7.94195458e-02,
         4.92859967e-02,  4.32612784e-02, -3.25758159e-02,
        -5.29966764e-02,  8.15892592e-02,  1.73796993e-02,
        -1.55277792e-02, -9.54639614e-02, -1.64624061e-02,
         4.38066153e-03, -7.02854842e-02,  8.67424533e-03,
         1.35337822e-02,  3.58423330e-02, -2.49423627e-02,
         7.55811185e-02,  2.23724078e-02, -5.77496924e-03,
         1.82581879e-03, -9.24453884e-02,  7.30333850e-0

In [10]:
# Add Feature Vectors to dataframe
df['Feature_Vector'] = dummy

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
# Export dataframe to CSV
df.to_csv(r'DS_25_Feature_Vectors.csv')

### Send Corpus as one complete list  - DOES NOT WORK

In [13]:
tempBatch = tempCorpus.tolist()

In [23]:
tempBatch[2]

"About Pivot Bio ---------------  Fueled by an innovative drive and a deep understanding of the soil microbiome, Pivot Bio is pioneering game-changing advances in agriculture. Our first commercial product harnesses the power of naturally-occurring microbes to provide nutrients to crops. We are dedicated to providing new sustainable ways for farmers to improve yield as they work to help feed the world's growing population.  The Field Technology team measures and models the performance of Pivot's microbial product. We aim to use this proprietary data to increase the value of our product. The Senior Data Scientist will analyze large, complex data sets to create and optimize predictive models using machine learning and other statistical tools. They will own the prototyping and early development of data products that can complement our microbial nitrogen fertilizer.  Location  Berkeley, CA  Responsibilities   Own the development of predictive analytics that quantify the impact and increase 

In [20]:
dummy_2 = []

with tf.Graph().as_default():
    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    messages = [tempBatch]
    output = embed(messages)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    t1 = time.time()
    message_embeddings = session.run(output)
    print(time.time() - t1)
    dummy_2.append(message_embeddings)

TypeError: Can't convert 'text': Shape TensorShape([Dimension(1), Dimension(25)]) is incompatible with TensorShape([Dimension(None)])

In [ ]:
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))
    dummy.append(message_embedding)